# Generate Images with Corel LoRA

This notebook generates images using LoRA weights trained on the Corel dataset.

## Instructions for Google Colab

1. Upload trained LoRA weights (`.safetensors` file) to Colab or Google Drive
2. Configure parameters in the "Configuration" section
3. Run all cells in order


## 1. Install Dependencies

**Install all required packages for GPU-accelerated generation:**


In [ ]:
# Install required dependencies for GPU generation
%pip install -q diffusers accelerate transformers torch torchvision

# Optional but recommended for GPU optimization
%pip install -q xformers  # Memory-efficient attention (optional)

print("="*60)
print("✓ DEPENDENCIES INSTALLED")
print("="*60)
print("Core packages: diffusers, accelerate, transformers")
print("Optimization: xformers (optional, saves memory)")
print("="*60)


## 2. Mount Google Drive

If your LoRA weights are in Google Drive, uncomment and run this cell:


In [ ]:
# Uncomment the following lines if you need to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted")


## 3. Imports


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from diffusers.utils import make_image_grid
from diffusers import EulerDiscreteScheduler
from datetime import datetime
from pathlib import Path
import re

print("Imports completed")


## 4. Configuration

**IMPORTANT:** Adjust these parameters according to your needs:


In [ ]:
# ===== CONFIGURATION =====
# EASILY CONFIGURABLE FOR LOCAL OR CLOUD EXECUTION

# Option 1: For Google Colab (uncomment and adjust path)
# BASE_DIR = Path('/content')
# BASE_DIR = Path('/content/drive/MyDrive/your_project_path')

# Option 2: For local execution
BASE_DIR = Path('.')

# Option 3: For other cloud services (adjust as needed)
# BASE_DIR = Path('/workspace')
# BASE_DIR = Path('/data')

print(f"✓ Base directory: {BASE_DIR.absolute()}")
print(f"✓ Working directory: {Path.cwd()}")

# Directory with LoRA weights
# If using Google Drive: "/content/drive/MyDrive/path/to/corel_models"
lora_dir = str(BASE_DIR / "corel_models")

# Specific LoRA file name (None = use the most recent)
lora_name = None  # Example: "lora_stable-diffusion-v1-5_rank4_s800_r512_DDPMScheduler_corel_all_20250101-120000.safetensors"

# Base Stable Diffusion model
pretrained_model = "runwayml/stable-diffusion-v1-5"

# Text prompt (None = use automatic prompts based on LoRA name)
prompt = None  # Example: "a photo of a royalguard"

# Generation parameters
num_images = 4
width = 512
height = 512
guidance_scale = 7.5
seed = 42

# Output directory
output_dir = str(BASE_DIR / "generated_images")

# ===== END CONFIGURATION =====

# Create output directory
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Verify CUDA availability and GPU info
if torch.cuda.is_available():
    device = "cuda:0"
    print(f"\n{'='*60}")
    print("✓ CUDA AVAILABLE - GPU OPTIMIZATION ENABLED")
    print(f"{'='*60}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"Device: {device}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"{'='*60}\n")
    
    # Set CUDA optimizations
    torch.backends.cudnn.benchmark = True  # Optimize for consistent input sizes
    torch.backends.cudnn.deterministic = False  # Allow non-deterministic for speed
    print("✓ CUDA optimizations enabled (cudnn.benchmark=True)\n")
else:
    device = "cpu"
    print("\n⚠ WARNING: CUDA not available! Generation will be VERY slow on CPU.")
    print("⚠ This notebook is optimized for GPU. Consider using a GPU-enabled environment.")
    print(f"⚠ Device: {device}\n")

print("="*60)
print("GENERATION CONFIGURATION")
print("="*60)
print(f"LoRA directory: {lora_dir}")
print(f"LoRA name: {lora_name if lora_name else 'Latest'}")
print(f"Base model: {pretrained_model}")
print(f"Prompt: {prompt if prompt else 'Auto (from LoRA name)'}")
print(f"Number of images: {num_images}")
print(f"Resolution: {width}x{height}")
print(f"Guidance scale: {guidance_scale}")
print(f"Seed: {seed}")
print("="*60)


## 5. Find LoRA File


In [ ]:
# Find LoRA file
lora_path = Path(lora_dir)

if lora_name:
    lora_file = lora_path / lora_name
    if not lora_file.exists():
        raise FileNotFoundError(f"LoRA file not found: {lora_file}")
    print(f"Using specific LoRA: {lora_name}")
else:
    # Find the most recent file
    lora_files = list(lora_path.glob("*.safetensors"))
    if not lora_files:
        raise FileNotFoundError(f"No LoRA files found in {lora_dir}")
    lora_file = sorted(lora_files, key=lambda x: x.stat().st_mtime)[-1]
    lora_name = lora_file.name
    print(f"Using most recent LoRA: {lora_name}")

print(f"Full file path: {lora_file}")


## 6. Load Model and LoRA


In [ ]:
# Load Stable Diffusion model
print("\nLoading Stable Diffusion model...")
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model,
    torch_dtype=torch.float16,
    safety_checker=None  # Remove safety checker to save VRAM
).to(device)

# Enable memory optimizations
pipe.enable_attention_slicing(slice_size=1)
pipe.enable_vae_tiling()
print("Memory optimizations enabled")

# Load LoRA weights
print(f"\nLoading LoRA: {lora_name}")
pipe.load_lora_weights(
    pretrained_model_name_or_path_or_dict=str(lora_path),
    weight_name=lora_name,
    adapter_name="corel_lora"
)
pipe.set_adapters(["corel_lora"], adapter_weights=[1.0])
print("LoRA loaded successfully")

# Configure scheduler
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
print("Scheduler configured")


## 7. Determine Prompts


In [ ]:
# Determine prompts based on LoRA name or use provided prompt
if prompt:
    prompts = [prompt]
else:
    # Extract information from LoRA filename
    if "class_" in lora_name:
        # Try to extract class number
        match = re.search(r'class_(\d+)', lora_name)
        if match:
            class_num = match.group(1)
            prompts = [f"a photo of a corel class {class_num}"]
        else:
            prompts = ["a photo of a corel image"]
    elif "corel_all" in lora_name:
        # Generic prompts for all-classes model
        prompts = [
            "a photo of a corel image",
            "a high quality corel image",
            "a detailed corel photograph"
        ]
    else:
        prompts = ["a photo of a corel image"]

print(f"Prompts determined: {len(prompts)}")
for i, p in enumerate(prompts):
    print(f"  {i+1}. {p}")


## 8. Generate Images


In [ ]:
# Generate images
all_images = []
all_prompts = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
negative_prompt = "low quality, blur, watermark, words, name, text"

for prompt_idx, current_prompt in enumerate(prompts):
    print(f"\n{'='*60}")
    print(f"GENERATING - Prompt {prompt_idx+1}/{len(prompts)}")
    print(f"{'='*60}")
    print(f"Prompt: {current_prompt}")
    print(f"Resolution: {width}x{height}")
    print(f"Number of images: {num_images}")
    print(f"Guidance scale: {guidance_scale}")
    print(f"Seed: {seed}")
    print(f"{'='*60}")
    
    # Generate images one at a time for better memory management
    images = []
    
    for i in range(num_images):
        print(f"\n[{i+1}/{num_images}] Generating image...")
        
        # Create generator on the correct device
        if device.startswith("cuda"):
            generator = torch.Generator(device=device)
        else:
            generator = torch.Generator()
        generator.manual_seed(seed + i)
        
        image = pipe(
            prompt=current_prompt,
            negative_prompt=negative_prompt,
            num_images_per_prompt=1,
            generator=generator,
            width=width,
            height=height,
            guidance_scale=guidance_scale
        ).images[0]
        
        images.append(image)
        
        # Save individual image
        safe_prompt = "".join(c for c in current_prompt[:30] if c.isalnum() or c in (' ', '-', '_')).strip()
        safe_prompt = safe_prompt.replace(' ', '_')
        img_path = Path(output_dir) / f"corel_{safe_prompt}_{timestamp}_{i+1:02d}.png"
        image.save(img_path)
        print(f"  Saved: {img_path.name}")
        
        # Clear cache between generations
        torch.cuda.empty_cache()
    
    all_images.extend(images)
    all_prompts.extend([current_prompt] * len(images))
    
    # Create grid for this prompt
    if len(images) > 1:
        grid = make_image_grid(images, cols=min(4, len(images)), rows=1)
        grid_path = Path(output_dir) / f"corel_grid_{safe_prompt}_{timestamp}.png"
        grid.save(grid_path)
        print(f"  Grid saved: {grid_path.name}")

print("\nGeneration completed")


## 9. Create Overall Grid and Visualize


In [ ]:
# Create overall grid if multiple prompts
if len(all_images) > 1:
    print(f"\n{'='*60}")
    print("Creating overall grid...")
    overall_grid = make_image_grid(all_images, cols=min(4, len(all_images)), rows=len(prompts))
    overall_grid_path = Path(output_dir) / f"corel_all_grid_{timestamp}.png"
    overall_grid.save(overall_grid_path)
    print(f"Overall grid saved: {overall_grid_path.name}")

# Display images in notebook
from IPython.display import display, Image as IPImage

print("\n" + "="*60)
print("GENERATED IMAGES")
print("="*60)

if len(all_images) > 1:
    display(overall_grid)
else:
    display(all_images[0])

print(f"\nTotal images generated: {len(all_images)}")
print(f"Output directory: {output_dir}")
print("\nGenerated files:")
for img_path in sorted(Path(output_dir).glob(f"corel_*_{timestamp}*.png")):
    print(f"  - {img_path.name}")
print("="*60)

# Cleanup
pipe.to("cpu")
torch.cuda.empty_cache()
print("\nCleanup completed")

# Optionally save to Google Drive
print("\nTo save to Google Drive, run:")
print("import shutil")
print(f"for img_path in Path('{output_dir}').glob('corel_*_{timestamp}*.png'):")
print("    drive_path = f'/content/drive/MyDrive/generated_images/{img_path.name}'")
print("    shutil.copy(img_path, drive_path)")
